In [ ]:
import os
import plotly.graph_objs as go
from plotly.offline import iplot
from pathlib import Path
import git

In [ ]:
%load_ext autoreload
%autoreload 2
from make_folder_dataset import MakeFolderDataset

## Load data

In [ ]:
def get_git_root():
    git_repo = git.Repo(os.getcwd(), search_parent_directories=True)
    return git_repo.git.rev_parse("--show-toplevel")

In [ ]:
instances = []
dataset_path = Path(get_git_root()) / "dataset"
for p in dataset_path.iterdir():
    if p.is_dir() and not p.name == "_ignore":
        instance = MakeFolderDataset(p.absolute())
        instance.extract_robot_data()
        instance.get_labels_all()
        instances.append(instance)

# Plot data

In [ ]:
target = ['etau_J0','etau_J1', 'etau_J2', 'etau_J3', 'etau_J4', 'etau_J5', 'etau_J6']
#target = ['e0','e1','e2','e3','e4','e5','e6']
#target = ['de0','de1','de2','de3','de4','de5','de6']

for i in target:
    for inst in instances:
        # label gets scaled otherwise measure and label are not visible properly on plot
        A = inst.df[i].max()-inst.df[i].min()
        inst.true_label['label_scaled']=inst.true_label['DATA0']*A + inst.df[i].min() 

        # use plotly to make interactive plots
        trace1 = go.Scatter(x=inst.true_label['time'], y=inst.true_label['label_scaled'], name='true label')
        trace2 = go.Scatter(x=inst.df['time'], y=inst.df[i], mode='lines', name='robot data')
        data = [trace1, trace2]
        layout = go.Layout(title=f'{i} (instance {os.path.basename(os.path.normpath(inst.path))})',
                        xaxis=dict(title='time(sec)'),
                        yaxis=dict(title='Y-axis'))
        fig = go.Figure(data=data, layout=layout)
        iplot(fig)